In [62]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Define the search query
location = input("Enter the location: ")
min_price = input("Enter the minimum price: ")
max_price = input("Enter the maximum price: ")
min_area = input("Enter the minimum area: ")
max_area = input("Enter the maximum area: ")
distance = input("Enter the maximum distance from location in km: ")
listing_type = input("Enter listing type (mieszkanie/dom/dzialka): ")
offer_type = input("Enter offer type (sprzedaz/wynajem): ")

# Start the Chrome driver and maximize the window
driver = webdriver.Chrome()
driver.maximize_window()

# Set the variable to check if the banner has been closed
is_banner_closed = False
listings_combined =[]


# Iterate through the pages and open each page in a new window
for page in range(1, 2): #page number
    # Construct the URL
    url =f"https://www.otodom.pl/pl/oferty/{offer_type}/{listing_type}/{location}?priceMin={min_price}&priceMax={max_price}&areaMin={min_area}&areaMax={max_area}&distanceRadius={distance}&page={page}&limit=72"

    # Open the URL in a new window
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(url)

    time.sleep(3)

    # Close Cookies banner
    if not is_banner_closed:
        try:
            cookie_banner = driver.find_element(By.ID,'onetrust-accept-btn-handler')
            cookie_banner.click()
            is_banner_closed = True
        except:
            pass

    # Scroll page down till the end
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Wait for the results to load
    #WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@role='navigation']")))


    # Scroll down to the end of the page
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(driver.page_source)

    # Find all the relevant information

    listings = soup.find_all("li", {'class': 'css-iq9jxc e1n6ljqa1'})
    listings_combined.append({'page': page, 'listings': listings})

# Close all windows and quit the driver
#driver.quit()    

In [67]:
listings_combined

[{'page': 1,
  'listings': [<li class="css-iq9jxc e1n6ljqa1" data-cy="listing-item"><a class="css-1up0y1q e1n6ljqa16" data-cy="listing-item-link" href="/pl/oferta/premium-standard-z-ogrodkiem-bez-prowizji-ID4kc0H"><aside class="css-qou0ww e1n6ljqa3"><picture><source data-cy="listing-item-image-source" media="(max-width: 768px)" srcset="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6Ijd6bnR4dmhlZmpubjMtQVBMIiwidyI6W3siZm4iOiJlbnZmcXFlMWF5NGsxLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.E0mTCcEbIhKnHN7OSOjs7WRURP0VPljjKp0c6jcGkbc/image;s=655x491;q=80"><source data-cy="listing-item-image-source" media="(min-width: 768px)" srcset="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6Ijd6bnR4dmhlZmpubjMtQVBMIiwidyI6W3siZm4iOiJlbnZmcXFlMWF5NGsxLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.E0mTCcEbIhKnHN7OSOjs7WRURP0VPljjKp0c6jcGkbc/image;s=1280x1024;q=80"><img alt="PREMIUM Standard z ogródkiem - BEZ PROWIZJI !" class="css-s6l5j9 e1xjv2fs0" height="210" loading="lazy" src="http

In [71]:
#this gets the Top 13 results only
data = []

for i in listings_combined:
    for record in listings:
    # Extract the link to the record
        link = record.find("a", {'class':'css-1up0y1q e1n6ljqa16'}).get("href")
        link = "https://www.otodom.pl/"+link

    # Extract the title
        property_title = record.find("h3", {'class' :'css-1mmyqk7 e1n6ljqa6'}).get("title")

    # Extract the Offerer
        offerer =record.find("p", {'class' :'css-1eeelu6 e17fvs873'}).text.strip()  

    # Extract the other details
        other_details =record.find("span", {'class' :'css-1em65uo ei6hyam2'}).text.strip()   


    # Extract the number of rooms, area, and price
        details = record.find_all("span", {'class':'css-1ntk0hg ei6hyam1'})
        rooms = ""
        area = ""
        price = ""
        for detail in details:
            if "pokoje" in detail.text:
                rooms = detail.text.strip().replace(" pokoje", "")
            elif "m²" in detail.text:
                area = detail.text.strip().replace(" m²", "")
            elif "zł/mc" in detail.text:
                price = detail.text.strip().replace("zł/mc", "")

        # Append the attributes to the data list
        data.append({
            "Link": link,
            "Property": property_title,
            "Address ": address,
            "Rooms": rooms,
            "Area (m²)": area,
            "Price (zł/mc)": price,
            "Offerer":offerer,
            "other Details": other_details
        })



# Create a dataframe from the data list
df = pd.DataFrame(data)

In [72]:
df

,Link,Property,Address,Rooms,Area (m²),Price (zł/mc),Offerer,other Details
0,https://www.otodom.pl//pl/oferta/premium-stand...,PREMIUM Standard z ogródkiem - BEZ PROWIZJI !,"Warszawa, Ursynów, Nugat 9",2,30,3500,Oferta prywatna,+ czynsz: 600 zł/miesiąc
1,https://www.otodom.pl//pl/oferta/2-pokojowe-mi...,2-pokojowe mieszkanie na Chodeckiej ),"Warszawa, Ursynów, Nugat 9",2,40,2300,Oferta prywatna,+ czynsz: 750 zł/miesiąc
2,https://www.otodom.pl//pl/oferta/mieszkanie-na...,Mieszkanie na wynajem w cichej okolicy,"Warszawa, Ursynów, Nugat 9",3,80,3400,Oferta prywatna,+ czynsz: 600 zł/miesiąc
3,https://www.otodom.pl//pl/oferta/apartament-mi...,Apartament Miasteczko Wilanów BEZPOŚREDNIO - 30m2,"Warszawa, Ursynów, Nugat 9",,30,3200,Oferta prywatna,+ czynsz: 580 zł/miesiąc
4,https://www.otodom.pl//pl/oferta/nowa-kawalerk...,Nowa kawalerka z garażem w pobliżu PKP Ursus,"Warszawa, Ursynów, Nugat 9",,31,2690,Oferta prywatna,+ czynsz: poszukaj w opisie
...,...,...,...,...,...,...,...,...
70,https://www.otodom.pl//pl/oferta/mieszkanie-pl...,Mieszkanie - Płochocińska - studenci - inni,"Warszawa, Ursynów, Nugat 9",,25,1500,Oferta prywatna,+ czynsz: 10 zł/miesiąc
71,https://www.otodom.pl//pl/oferta/mieszkanie-do...,Mieszkanie do wynajęcia Otwock park Hallera Mo...,"Warszawa, Ursynów, Nugat 9",2,52,2200,Oferta prywatna,+ czynsz: 520 zł/miesiąc
72,https://www.otodom.pl//pl/oferta/mokotow-mordo...,"Mokotów , Mordor, Galeria Mokotów , kawalerka","Warszawa, Ursynów, Nugat 9",,32,2000,Biuro nieruchomości,+ czynsz: 300 zł/miesiąc
73,https://www.otodom.pl//pl/oferta/2-pokoje-okol...,2 pokoje okolice CH.Arkadia,"Warszawa, Ursynów, Nugat 9",2,41,2950,Biuro nieruchomości,+ czynsz: poszukaj w opisie


In [36]:
#df["Rooms"] = df["Rooms"].astype(float)
#df["Area (m²)"] = df["Area (m²)"].astype(float)
df["Price (zł/mc)"] = df["Price (zł/mc)"].astype(float)
print(df.dtypes)

Link              object
Property          object
Address           object
Rooms             object
Area (m²)         object
Price (zł/mc)    float64
dtype: object


In [39]:
df.Link[1]

'https://www.otodom.pl//pl/oferta/zielen-za-oknem-ID4kGCY'

In [53]:
is_banner_closed = False
for i in df['Link'].iloc[:1]:
    driver = webdriver.Chrome()
    driver.get(i)

    # Close Cookies banner
    if not is_banner_closed:
        try:
            cookie_banner = driver.find_element(By.ID,'onetrust-accept-btn-handler')
            cookie_banner.click()
            is_banner_closed = True
        except:
            pass    

    time.sleep(1)

    # Locate the iframe element using its CSS selector
    
    iframe = driver.find_element(By.XPATH,"//div[@id='map']")

    # Switch to the iframe
    #driver.switch_to.frame(iframe)

    # Locate the position marker using its XPATH
    position_marker = driver.find_element(By.XPATH, "//map[@name='gmimap0']")

    # Extract the position of the marker
    lat = position_marker.get_attribute('data-lat')
    lng = position_marker.get_attribute('data-lng')
    
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//map[@name="gmimap0"]"}
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x00EDDCE3]
	(No symbol) [0x00E739D1]
	(No symbol) [0x00D84DA8]
	(No symbol) [0x00DB019F]
	(No symbol) [0x00DB03AB]
	(No symbol) [0x00DDEE62]
	(No symbol) [0x00DCAF14]
	(No symbol) [0x00DDD57C]
	(No symbol) [0x00DCACC6]
	(No symbol) [0x00DA6F68]
	(No symbol) [0x00DA80CD]
	GetHandleVerifier [0x01153832+2506274]
	GetHandleVerifier [0x01189794+2727300]
	GetHandleVerifier [0x0118E36C+2746716]
	GetHandleVerifier [0x00F86690+617600]
	(No symbol) [0x00E7C712]
	(No symbol) [0x00E81FF8]
	(No symbol) [0x00E820DB]
	(No symbol) [0x00E8C63B]
	BaseThreadInitThunk [0x75B87D69+25]
	RtlInitializeExceptionChain [0x7710B74B+107]
	RtlClearBits [0x7710B6CF+191]
